# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal is to create a Python script that will accept the value of my portfolio and tell me how many shares of each S&P 500 constituent I should purchase to get an equal-weight version of the index fund.

## Library Imports




In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks



In [2]:
stocks = pd.read_csv(r"C:\Users\conta\OneDrive\Desktop\Project Pro\algorithmic-trading-python-master\algorithmic-trading-python-master\starter_files\sp_500_stocks.csv")

## Acquiring an API Token



In [3]:
from secret import IEX_CLOUD_API_TOKEN

## Making Our First API Call





In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(api_url)
type(data)
#print(data.status_code)
print(data)

https://sandbox.iexapis.com/stable/stock/AAPL/quote/?token=Tpk_059b97af715d417d9f49f50b51b1c448
{'avgTotalVolume': 75691957, 'calculationPrice': 'close', 'change': 0, 'changePercent': 0, 'close': 0, 'closeSource': 'fifolcai', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': None, 'highTime': None, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 155.75, 'iexCloseTime': 1714951939803, 'iexLastUpdated': 1689175886504, 'iexMarketPercent': 0.0021215699478335692, 'iexOpen': 155.29, 'iexOpenTime': 1739423521900, 'iexRealtimePrice': 157, 'iexRealtimeSize': 10, 'iexVolume': 2325, 'lastTradeTime': 1700176487499, 'latestPrice': 156.53, 'latestSource': 'Close', 'latestTime': 'September 13, 2022', 'latestUpdate': 1688275319812, 'latestVolume': None, 'low': 0, 'lowSource': N

## Parsing Our API Call



In [5]:
price = data['latestPrice']
m_c = data['marketCap']/1000000000000
print(price)
print(m_c)

156.53
2.518437870921


## Adding Our Stocks Data to a Pandas DataFrame



In [6]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy',]
df = pd.DataFrame(columns= my_columns)
df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [7]:
df.append(
pd.Series([
    symbol,
    price,
    m_c,
    'N/A'
],index = my_columns),ignore_index = True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,156.53,2.518438,N/A


## Looping Through The Tickers in Our List of Stocks



In [8]:
df =pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker'][:5]:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    df = df.append(pd.Series([
        stock,
        data['latestPrice'],
        data['marketCap'],
        'N/A'
    ], index = my_columns), ignore_index = True)


In [9]:
df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,156.49,2546957485863,N/A
1,AAL,155.67,2573615589051,N/A
2,AAP,155.93,2484505911849,N/A
3,AAPL,159.33,2586939263234,N/A
4,ABBV,159.33,2531598669434,N/A


## Using Batch API Calls to Improve Performance


In [10]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [11]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
df = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if symbol == 'DISCA' or symbol == 'HFC' or symbol == 'VIAC' or symbol == 'WLTW': # there are some errors with these stocks
            continue
        df = df.append(
            pd.Series(
            [
                symbol, 
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['marketCap'],
                'N/A'
            ],
            index = my_columns),
            ignore_index = True
        )

## Calculating the Number of Shares to Buy



In [12]:
portfolio_size = input('Enter the value of your portfolio:')

try:
    val = float(portfolio_size)
except ValueError:
        print("That's not a number! \nPlease use numbers only:")
        portfolio_size = input('Enter the value of your portfolio:')
        val = float(portfolio_size)
    

Enter the value of your portfolio:
That's not a number! 
Please use numbers only:
Enter the value of your portfolio:1000000


In [13]:
position_size = val/len(df.index)
for i in range(0, len(df.index)):
    df.loc[i,'Number of Shares to Buy'] = math.floor(position_size/df.loc[i, 'Stock Price'])
#number_of_apple_shares =position_size/155
#print(math.floor(number_of_apple_shares))
df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,138.74,39795032517,14
1,AAL,13.70,9327236376,145
2,AAP,178.27,10780991421,11
3,AAPL,156.65,2511313268741,12
4,ABBV,144.90,253383723258,13
...,...,...,...,...
496,YUM,115.90,32425229277,17
497,ZBH,117.80,25331835326,16
498,ZBRA,306.50,15852771471,6
499,ZION,59.50,8969627248,33


## Formatting Our Excel Output



### Initializing our XlsxWriter Object

In [14]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='Recommended Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [15]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format  = writer.book.add_format(
{   
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
})
dollar_format = writer.book.add_format(
{   'num_format': '$0.00',
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
})
integer_format = writer.book.add_format(
{   'num_format': '0',
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
})

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.


In [16]:
#writer.sheets['Recommended Trades'].set_column('A:A', #This tells the method to apply the format to column B
                     #18, #This tells the method to apply a column width of 18 pixels
                     #string_format #This applies the format 'string_template' to the column)
                    
#writer.sheets['Recommended Trades'].set_column('B:B',18,string_format)
#writer.sheets['Recommended Trades'].set_column('C:C',18,string_format)
#writer.sheets['Recommended Trades'].set_column('D:D',18,string_format)
#writer.save

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [17]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}',18,column_formats[column][1])
    
writer.save()   

## Saving Our Excel Output

Saving our Excel file is very easy: